In [1]:
# https://wikidocs.net/233348

In [2]:
# pip install langchain

In [3]:
# pip install -U langchain-ollama

In [4]:
# pip install langchain_teddynote
#  ollama create llama-3.1-70b-instruct-lorablated.Q4_K_M:latest -f Modelfile

In [1]:

import warnings
import langchain
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore", category=DeprecationWarning)
# https://wikidocs.net/233348

In [2]:
df = pd.read_csv('../../data/q_output.csv')

In [3]:
llm = OllamaLLM(model="llama-3.1-70b-instruct-lorablated.Q4_K_M:latest")

In [4]:
df.shape[0]

44

In [5]:
df=df[['question', 'answer']]

In [6]:
population = np.arange(44)
np.random.seed(19)
first_ann_q_id = np.random.choice(population, size=20, replace=False)


In [7]:
first_ann_q_id

array([14, 25, 13, 21, 43,  6, 41,  1,  0, 32])

In [8]:
len(first_ann_q_id[:-1])

9

In [9]:
examples = []
for idx in first_ann_q_id[:-5]:
# for idx in [19]:
    temp_dict = {"question" : str(df.iloc[idx, 0]),
                 "answer"   : str(df.iloc[idx, 1])}
    examples.append(temp_dict)

In [10]:
first_ann_q_id

array([14, 25, 13, 21, 43,  6, 41,  1,  0, 32])

In [11]:
example_prompt = PromptTemplate.from_template(
    "Question: {question} *||* Answer:{answer}"
)

print(example_prompt.format(**examples[0]))

Question: Here is the question about software development. The title of the question is "Expired tokens are not deleted after expiration in django-rest-knox 4.1.0". and body of the question is "I am using django-rest-knox 4.1.0 . In its documentation it says that expired tokens are deleted automatically. But it is not deleting the tokens.




In settings.py I gave 10 minutes for expiration of token (for testing purposes). ";TOKEN_TTL";: timedelta(minutes=10)
I check the database after that time, they are not deleted. pgadmin knox token table Also I try to send request with those expired tokens, the respond is successful.
". Please answer briefly about the "difficulty class" and "general ruleset" and "grenular breakdown" for the question *||* Answer:Difficulty class : Advanced Level Knowledge | General rule set : A2-System Architecture | Granular breakdown : 분산시스템/확장성/보안


In [12]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {question} *||* Answer:",
    input_variables=["question"],
)

In [13]:
first_ann_q_id[-1]

32

In [14]:
question = df.iloc[first_ann_q_id[-1], 0]
final_prompt = prompt.format(question=question)
print(final_prompt)


Question: Here is the question about software development. The title of the question is "Expired tokens are not deleted after expiration in django-rest-knox 4.1.0". and body of the question is "I am using django-rest-knox 4.1.0 . In its documentation it says that expired tokens are deleted automatically. But it is not deleting the tokens.




In settings.py I gave 10 minutes for expiration of token (for testing purposes). ";TOKEN_TTL";: timedelta(minutes=10)
I check the database after that time, they are not deleted. pgadmin knox token table Also I try to send request with those expired tokens, the respond is successful.
". Please answer briefly about the "difficulty class" and "general ruleset" and "grenular breakdown" for the question *||* Answer:Difficulty class : Advanced Level Knowledge | General rule set : A2-System Architecture | Granular breakdown : 분산시스템/확장성/보안

Question: Here is the question about software development. The title of the question is "Add error handling in Pytho

In [15]:
chain = prompt | llm

# chain 호출
response = chain.invoke({"question": question})

In [16]:
print(response)

Difficulty class : Intermediate Level  | General rule set : I2-Performance | Granular breakdown : 성능 최적화/데이터 구조


In [17]:
df.iloc[first_ann_q_id[-1], 1]

'Difficulty class : Intermediate Level  | General rule set : I2-Performance | Granular breakdown : 성능 최적화/자료구조'